# Generate stores & customers for the demo

## Setup the environment

### Install packages

> **Note:** After running the pip command you should restart the Jupyter kernel.<br>
> To restart the kernel, click on the kernel-restart button in the notebook's action toolbar (the refresh icon next to the **Code** button).

In [46]:
!pip install faker
!pip install pymorton
!pip install v3io
!pip install v3io_frames

### imports

In [47]:
import logging
import os
import pickle
import random
import itertools

# Demo
import pandas as pd
import pymorton as pm
from faker import Faker
from faker.providers import BaseProvider

# DB
import v3io as v3c
import sys
sys.path.append('../../')
import v3io_frames_local as v3f

### Define environment variables

In [48]:
# Stores
NUMBER_OF_STORES=50
STORES_TABLE=os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'stores')
ACCURACY=20
BASE_ACCURACY=14

# Customers
NUMBER_OF_CUSTOMERS=5000
CUSTOMERS_TABLE=os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'customers')
os.environ['CUSTOMERS_DATA_FILE_DIR'] = os.path.join(os.getcwd(), 'data')
os.environ['CUSTOMERS_DATA_FILE_PATH'] = os.path.join(os.environ['CUSTOMERS_DATA_FILE_DIR'], 'data_train_n.csv')
!mkdir -p ${CUSTOMERS_DATA_FILE_DIR}

In [49]:
!curl -L "iguazio-sample-data.s3.amazonaws.com/data_train_n.csv" > ${CUSTOMERS_DATA_FILE_PATH}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  399M  100  399M    0     0  5304k      0  0:01:17  0:01:17 --:--:-- 4795k


In [50]:
!ls -lah ${CUSTOMERS_DATA_FILE_DIR}

total 0
-rw-r--r-- 1 root nogroup 400M Jan 10 18:17 data_train_n.csv


### Open DB client

In [51]:
client = v3f.Client('framesd:8081', container='users')

## Generate Stores

### Location provider

In [52]:
class LocationProvider(BaseProvider):
    '''
    Creates locations within base_location

    Uses QuadTree for Geohashing
        @{http://tech.taskrabbit.com/blog/2015/06/09/elasticsearch-geohash-vs-geotree/}
        @{http://mapzen.github.io/leaflet-spatial-prefix-tree/}
        @{http://blog.notdot.net/2009/11/Damn-Cool-Algorithms-Spatial-indexing-with-Quadtrees-and-Hilbert-Curves}
    '''    
    def location(self, location_base: str, base_acc=10, acc=20):
        coordinates = location_base[:base_acc]
        for i in range(acc-len(coordinates)):
            coordinates += str(random.randint(0, 3))
        return coordinates

Add the location provider as a faker provider

In [53]:
faker = Faker()
faker.add_provider(LocationProvider)

### Define scenario environment (locations)

In [54]:
london_city = (51.514926, -0.089580)
london_city_south = (51.501593, -0.094942)
london_west = (51.512309, -0.128966)
london_south_west = (51.495022, -0.162268)

london_coordinates = []
london_coordinates.append(london_city)
london_coordinates.append(london_city_south)
london_coordinates.append(london_west)
london_coordinates.append(london_south_west)

london_coordinates_qt = list(map(lambda cooridnate: pm.interleave_latlng(*cooridnate), london_coordinates))
london_coordinates_qt

['03311311313011311011000321002320',
 '03311311311233323013031101320003',
 '03311311313010023000032330133111',
 '03311311311222300331010333220231']

### Define Store class

In [55]:
class Store(object):
    def __init__(self, store_id: int, name: str, location: str):
        self.store_id = store_id if store_id is not None else random.randint(1, int(os.genev('NUMBER_OF_STORES', 500)))
        self.location = location
        (self.lat, self.long) = pm.deinterleave_latlng(location)
        self.store_name = name
        self.customers = 0
    
    def json(self):
        json_store = {
            'id': self.store_id,
            'name': self.store_name,
            'location': self.location,
            'latitude': self.lat,
            'longitude': self.long,
            'customers': self.customers
        }
        return json_store

### Create stores

In [56]:
def create_stores(faker, number_of_stores: int=500):
    remaining = number_of_stores
    stores = []
    while remaining > 0:
        # Create stores
        stores_batch = [Store(faker.msisdn(), faker.company(), faker.location(random.choice(london_coordinates_qt), BASE_ACCURACY, ACCURACY)).json() 
              for i in range(remaining)]
        
        # Count for duplicate locations
        locations = list(map(lambda store: store['location'], stores_batch))
        locations = list(dict.fromkeys(locations))
        remaining -= len(locations)
        
        # Update stores list
        stores.append(stores_batch)

    stores = list(itertools.chain.from_iterable(stores))
    stores = pd.DataFrame.from_records(stores)
    stores = stores.set_index(['location'])
    stores = stores[~stores.index.duplicated(keep='first')]
    return stores

In [57]:
client.delete('kv', STORES_TABLE, if_missing=1)

In [58]:
stores = create_stores(faker, NUMBER_OF_STORES)
stores.head(5)

,customers,id,latitude,longitude,name
location,,,,,
03311311311222323311,0,4180725720799,51.502533,-0.159645,"Arnold, Costa and Taylor"
03311311311233032331,0,2556497132084,51.492233,-0.101967,Nelson-Barrera
03311311311233133201,0,4166647808518,51.491547,-0.090294,Higgins-Sanchez
03311311313011202200,0,4613451694822,51.519012,-0.109863,Sloan-Ortiz
03311311311222012300,0,2898223416207,51.486053,-0.168915,Nichols-Gilmore


### Upload to DB 

In [59]:
# client.delete('kv', STORES_TABLE)
client.write('kv', STORES_TABLE, stores)

In [60]:
client.read('kv', STORES_TABLE).head(5)

,customers,id,latitude,longitude,name
location,,,,,
03311311311233032331,0,2556497132084,51.492233,-0.101967,Nelson-Barrera
03311311313011102012,0,6253000155459,51.506996,-0.098190,Jenkins-Ramos
03311311311222310123,0,1893148783613,51.493950,-0.157585,Garza LLC
03311311311233203000,0,2782830086037,51.495667,-0.107117,"Randolph, Collier and Rice"
03311311313010333001,0,2549662772327,51.523132,-0.112267,"Suarez, Mueller and Greene"


## Generate customers

In [61]:
class Customer(object):
    def __init__(self, id=0, location='0', products=""):
        self.id = id
        self.location = location
        self.products = products
        
    def json(self):
        json_customer = {
            'id': self.id,
            'location': self.location,
            'products': self.products
        }
        return json_customer

In [62]:
def create_customers(customers_data: pd.DataFrame, number_of_customers: int):
    available_customer = list(customers_data.user_id)
    customers = [Customer(id=i,
                          products=customers_data.loc[customers_data['user_id'] == random.choice(available_customer)]\
                                    .drop(['user_id', 'Unnamed: 0'], axis=1)\
                                    .to_json()).json() for i in range(number_of_customers)]
    customers = pd.DataFrame.from_records(customers)
    customers.set_index(['id'])
    
    # Verify string length for KV
    customers['verification'] = customers['products'].apply(lambda x: len(x.encode('utf-8')))
    customers = customers.loc[customers['verification'] <= 61000]
    customers = customers.drop(columns='verification')
    
    # Fix index
    customers = customers.reset_index(drop=True)
    return customers

In [63]:
# Load customers seed data
customers_data = pd.read_csv(os.environ['CUSTOMERS_DATA_FILE_PATH'])

# Create actual customers
customers = create_customers(customers_data, NUMBER_OF_CUSTOMERS)
customers.head(5)

,id,location,products
0,0,0,"{""user_product_reordered_ratio"":{""147444"":1.0,..."
1,1,0,"{""user_product_reordered_ratio"":{""163784"":1.0,..."
2,2,0,"{""user_product_reordered_ratio"":{""8373"":1.0,""7..."
3,3,0,"{""user_product_reordered_ratio"":{""154069"":1.0,..."
4,4,0,"{""user_product_reordered_ratio"":{""40121"":1.0,""..."


## Upload to DB

In [64]:
client.delete('kv', CUSTOMERS_TABLE, if_missing=1)
client.write('kv', CUSTOMERS_TABLE, dfs=customers)

In [65]:
print(f'Customers table sized: {client.read("kv", CUSTOMERS_TABLE).shape}')
client.read('kv', CUSTOMERS_TABLE).head(5)

Customers table sized: (4978, 3)


,id,location,products
idx,,,
101,101,0,"{""user_product_reordered_ratio"":{""336520"":1.0,..."
1686,1697,0,"{""user_product_reordered_ratio"":{""24495"":1.0,""..."
1848,1861,0,"{""user_product_reordered_ratio"":{""53188"":1.0,""..."
4887,4909,0,"{""user_product_reordered_ratio"":{""475299"":1.0,..."
2901,2917,0,"{""user_product_reordered_ratio"":{""46623"":1.0,""..."


## Generate predictions

In [66]:
# client.delete('tsdb', 'predictions', if_missing=1)
client.create('tsdb', os.getenv('V3IO_USERNAME'), 'iguazio', 'predictions', rate='1/s', if_exists=1)

TypeError: create() got multiple values for argument 'if_exists'